In [ ]:
"""
File: draw_segmentation_boundaries_napari.ipynb
Author: Clinton Reid
Description: creates and saves images (ch1 images in this case) overlaid with nuclear outlines and points representing
            the centre of said nuclei. assumes this notebook is in the same folder as the folder containing rounds imaging data folders.
"""

In [2]:
# built-in imports

import os
from glob import glob

# libraries
import pandas as pd
import numpy as np
import tifffile as tiff
import napari
from skimage.io import imsave

In [4]:
nuclear_channel = 5
view_channel = 1 # channel image to be shown
current_folder = os.getcwd()
image_sets_folder = os.path.join(current_folder, 'Images')

In [ ]:
image_sets = os.listdir(image_sets_folder)

viewer = napari.Viewer()

for image_set in image_sets:
    # filters out all files (anything with a '.' in the name likely has some file extension) in this folder
    if '.' not in image_set:
        image_set_folder = os.path.join(image_sets_folder, image_set)
        nuclear_mask, ch1_image = None, None
        
        # look only through previously generated cellpose outputs
        for file in glob(image_set_folder + '/*_seg.npy'):

            if int(file.split('h')[1][0]) == nuclear_channel: # find the nuclear output using file naming convention
                dat = np.load(file, allow_pickle=True).item() # then load the outlines
                nuclear_mask = dat['outlines'] 

        for file in glob(image_set_folder + '/*.tiff'):
            # we can find the desired image (ch1) in a similar way
            if int(file.split('h')[1][0]) == view_channel:
                ch1_image = tiff.imread(file)

        nucleus_locations = pd.read_csv(os.path.join(image_set_folder, 'nuc_center.csv'))
        
        viewer.add_image(ch1_image, name='ch1_image')

        if nuclear_mask is not None:

            viewer.add_labels(nuclear_mask, num_colors=1, name='nuclear_mask', opacity=0.7) # make all nuclei same color

        viewer.add_points(nucleus_locations[['centroid-0', 'centroid-1']], name='nuclei_locations', size=5, face_color='blue', edge_color='blue')
    
        screenshot = viewer.screenshot()
        ss_path = os.path.join(image_set_folder, 'nuclear_outlines_and_centers.png')
        imsave(ss_path, screenshot, check_contrast=False)

viewer.close()
